## Lib

In [5]:
import pandas as pd
import numpy as np

from sktime.classifiers.distance_based import KNeighborsTimeSeriesClassifier
from sktime.classifiers.compose import TimeSeriesForestClassifier
from sktime.classifiers.dictionary_based.boss import BOSSEnsemble

from sktime.classifiers.compose import ColumnEnsembleClassifier

from sktime.utils.load_data import from_long_to_nested
from sklearn.metrics import accuracy_score

## Data

In [6]:
full_array = np.load('data/Foch_transplant.npy', allow_pickle=True)
col = np.load('data/Foch_transplant_name_col.npy', allow_pickle=True)

In [7]:
full_array.shape

(330, 71)

In [8]:
df = pd.DataFrame(full_array, columns=col)

# Sktime

In [9]:
df.columns

Index(['Age_donor', 'Aspirations_donor', 'BMI_donor', 'Donneur_CPT',
       'Insuffisance_renale', 'LAS', 'PAPS', 'PFO', 'PF_donor', 'Poids',
       'Poids_donor', 'RX_donor', 'Sex_donor', 'Tabagisme_donor', 'Taille',
       'Taille_donor', 'age', 'body_mass_index', 'diabetes', 'id_patient',
       'other_organ_transplantation', 'oto_score', 'pathologie',
       'plasmapherese', 'preoperative_ECMO', 'preoperative_ICU',
       'preoperative_mechanical_ventilation',
       'preoperative_pulmonary_hypertension', 'preoperative_vasopressor',
       'retransplant', 'sexe', 'super_urgence', 'thoracic_surgery_history',
       'time_on_waiting_liste', 'transplanted_twice_during_study_period',
       'target', 'length_op', 'start_operation_year', 'start_operation_month',
       'start_operation_day', 'ends_operation_year', 'ends_operation_month',
       'ends_operation_day', 'B.I.S', 'BIS SR', 'DC', 'ETCO2', 'FC', 'FR',
       'FiO2', 'PAPdia', 'PAPmoy', 'PAPsys', 'PASd', 'PASm', 'PASs',
       

In [10]:
len(['B.I.S', 'BIS SR', 'DC', 'ETCO2', 'FC', 'FR',
       'FiO2', 'PAPdia', 'PAPmoy', 'PAPsys', 'PASd', 'PASm', 'PASs',
       'PEEPtotal', 'PNId', 'PNIm', 'PNIs', 'Pmax', 'Pmean', 'SpO2',
       'SvO2 (m)', 'Temp', 'VT', 'SpO2_by_FiO2', 'PASs_minus_PASd',
       'PAPsys_minus_PAPdia', 'declampage_cote1_done',
       'declampage_cote2_done'])

28

In [11]:
%%time
dic1 = {'case_id':[], 'dim_id':[], 'reading_id':[], 'value':[]}
nb_max = df['SpO2_by_FiO2'].apply(len).max()
for k, col in enumerate(['B.I.S', 'BIS SR', 'DC', 'ETCO2', 'FC', 'FR',
       'FiO2', 'PAPdia', 'PAPmoy', 'PAPsys', 'PASd', 'PASm', 'PASs',
       'PEEPtotal', 'PNId', 'PNIm', 'PNIs', 'Pmax', 'Pmean', 'SpO2',
       'SvO2 (m)', 'Temp', 'VT', 'SpO2_by_FiO2', 'PASs_minus_PASd',
       'PAPsys_minus_PAPdia', 'declampage_cote1_done',
       'declampage_cote2_done']):
    
    for i, arr in enumerate(df[col]):
        l = len(arr)
        for j, val in enumerate(arr):

            dic1['case_id']+= [i]
            dic1['dim_id']+= [k]
            dic1['reading_id']+= [j]
            dic1['value']+= [val]
        # get ts same length
        for h in range(nb_max-l):
            dic1['case_id']+= [i]
            dic1['dim_id']+= [k]
            dic1['reading_id']+= [l+h]
            dic1['value']+= [0]

Wall time: 6.05 s


In [12]:
ddff1 = pd.DataFrame(dic1)

In [13]:
%%time
X_nested1 = from_long_to_nested(ddff1)

X_nested1.columns = ['B.I.S', 'BIS SR', 'DC', 'ETCO2', 'FC', 'FR',
       'FiO2', 'PAPdia', 'PAPmoy', 'PAPsys', 'PASd', 'PASm', 'PASs',
       'PEEPtotal', 'PNId', 'PNIm', 'PNIs', 'Pmax', 'Pmean', 'SpO2',
       'SvO2 (m)', 'Temp', 'VT', 'SpO2_by_FiO2', 'PASs_minus_PASd',
       'PAPsys_minus_PAPdia', 'declampage_cote1_done',
       'declampage_cote2_done']

Wall time: 23min 5s


In [8]:
X_nested1.head()

,B.I.S,BIS SR,DC,ETCO2,FC,FR,FiO2,PAPdia,PAPmoy,PAPsys,...,Pmean,SpO2,SvO2 (m),Temp,VT,SpO2_by_FiO2,PASs_minus_PASd,PAPsys_minus_PAPdia,declampage_cote1_done,declampage_cote2_done
0,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 118.0 1 112.0 2 117.0 3 ...,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 21.0 3 ...,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,...,0 0.0 1 0.0 2 1.0 3 1....,0 99.0 1 99.0 2 97.0 3 ...,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 1.000000 1 1.000000 2 4.61...,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....
1,0 20.414414 1 20.414414 2 20...,0 0.695961 1 0.695961 2 0.69...,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 154.0 1 146.0 2 141.0 3 ...,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 ...,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,...,0 0.0 1 0.0 2 0.0 3 0....,0 99.0 1 100.0 2 100.0 3 ...,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 1.0 1 1.0 2 1.0 3 1....,0 67.0 1 59.0 2 62.0 3 ...,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....
2,0 20.414414 1 20.414414 2 20...,0 0.695961 1 0.695961 2 0.69...,0 2.229983 1 2.229983 2 2.22...,0 0.0 1 0.0 2 0.0 3 0....,0 111.0 1 122.0 2 112.0 3 ...,0 0.0 1 0.0 2 0.0 3 0....,0 99.0 1 99.0 2 99.0 3 ...,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,...,0 1.0 1 1.0 2 1.0 3 1....,0 89.0 1 90.0 2 88.0 3 ...,0 46.775341 1 46.775341 2 46...,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 0.898990 1 0.909091 2 0.88...,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....
3,0 20.414414 1 20.414414 2 20...,0 0.695961 1 0.695961 2 0.69...,0 2.229983 1 2.229983 2 2.22...,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 97.0 2 94.0 3 ...,0 0.0 1 0.0 2 0.0 3 0....,0 101.0 1 101.0 2 101.0 3 ...,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,...,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 84.0 2 0.0 3 ...,0 46.775341 1 46.775341 2 46...,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 0.000000 1 0.831683 2 0.00...,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....
4,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 110.0 2 111.0 3 ...,0 0.0 1 1.0 2 0.0 3 0....,0 23.0 1 22.0 2 23.0 3 ...,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,...,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 95.0 2 96.0 3 ...,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 0.000000 1 4.318182 2 4.17...,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....,0 0.0 1 0.0 2 0.0 3 0....


In [9]:
X_nested1.shape

(330, 28)

#### Sample of the base

#### 40 train 30 test

In [14]:
%%time
y_tr = df['target'][:40].ravel().astype('<U1')
y_te = df['target'][30:].ravel().astype('<U1')

result_knn = []
for col in X_nested1.columns:
    x_tr = X_nested1[:40][[col]]
    x_te = X_nested1[30:][[col]]
    
    knn = KNeighborsTimeSeriesClassifier(metric='dtw')
    knn.fit(x_tr, y_tr)
    
    s = knn.score(x_te, y_te)
    print(col + " : " + str(s))
    result_knn += [s]

B.I.S : 0.46
BIS SR : 0.51
DC : 0.51
ETCO2 : 0.58
FC : 0.48
FR : 0.47
FiO2 : 0.55
PAPdia : 0.54
PAPmoy : 0.53
PAPsys : 0.5
PASd : 0.54
PASm : 0.5
PASs : 0.58
PEEPtotal : 0.46
PNId : 0.48
PNIm : 0.51
PNIs : 0.45
Pmax : 0.52
Pmean : 0.5
SpO2 : 0.65
SvO2 (m) : 0.44
Temp : 0.49
VT : 0.46
SpO2_by_FiO2 : 0.58
PASs_minus_PASd : 0.59
PAPsys_minus_PAPdia : 0.53
declampage_cote1_done : 0.58
declampage_cote2_done : 0.58
Wall time: 1h 54min 58s


#### 100 train 100 test

In [16]:
%%time
y_tr = df['target'][:100].ravel().astype('<U1')
y_te = df['target'][230:].ravel().astype('<U1')

result_knn = []
for col in X_nested1.columns:
    x_tr = X_nested1[:100][[col]]
    x_te = X_nested1[230:][[col]]
    
    knn = KNeighborsTimeSeriesClassifier(metric='dtw')
    knn.fit(x_tr, y_tr)
    
    s = knn.score(x_te, y_te)
    print(col + " : " + str(s))
    result_knn += [s]

KeyboardInterrupt: 

##### Full

In [14]:
%%time
y_tr = df['target'][:230].ravel().astype('<U1')
y_te = df['target'][230:].ravel().astype('<U1')

result_knn = []
for col in X_nested1.columns:
    x_tr = X_nested1[:230][[col]]
    x_te = X_nested1[230:][[col]]
    
    knn = KNeighborsTimeSeriesClassifier(metric='dtw')
    knn.fit(x_tr, y_tr)
    
    s = knn.score(x_te, y_te)
    print(col + " : " + str(s))
    result_knn += [s]

B.I.S : 0.51
BIS SR : 0.46
DC : 0.47
ETCO2 : 0.61
FC : 0.56
FR : 0.54
FiO2 : 0.52
PAPdia : 0.55
PAPmoy : 0.51
PAPsys : 0.56
PASd : 0.51
PASm : 0.52
PASs : 0.51
PEEPtotal : 0.48
PNId : 0.48
PNIm : 0.45
PNIs : 0.46
Pmax : 0.59
Pmean : 0.5
SpO2 : 0.63
SvO2 (m) : 0.47
Temp : 0.47
VT : 0.48
SpO2_by_FiO2 : 0.62
PASs_minus_PASd : 0.57
PAPsys_minus_PAPdia : 0.51
declampage_cote1_done : 0.58
declampage_cote2_done : 0.58
Wall time: 4h 17min 28s


In [ ]:
BIS SR : 0.46 * 
ETCO2 : 0.61 * 
FC : 0.56 * 
FR : 
FiO2 : 0.52 * 
PAPsys : 0.56 * 
PASm : 0.52 * 
PEEPtotal : 0.48 * 
SpO2 : 0.63 * 
SpO2_by_FiO2 : 0.62 * 